In [10]:
import sys
sys.path.append('./Humpback-Whale-Identification-1st/')
import pandas as pd
import numpy as np
import os
import torch
from tqdm import tqdm
from models import *
from dataSet import *
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
import shutil
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
num_TTA = 2
import joblib
encoder = joblib.load('./le.pkl')

In [ ]:
checkPoint_start = 0
fold_index = 1
model_name = 'senet154'

In [3]:
names_test = os.listdir('../data/test_images-128-128')
batch_size = 16

In [8]:
CONFIG = {"seed": 2022,
          "epochs": 3,
          "img_size": 128,
          "model_name": "tf_efficientnet_b0",
          "num_classes": 15587,
          "train_batch_size": 64,
          "valid_batch_size": 128,
          "learning_rate": 1e-4,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,
          "T_max": 500,
          "weight_decay": 1e-6,
          "n_fold": 5,
          "n_accumulate": 1,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          # ArcFace Hyperparameters
          "s": 30.0, 
          "m": 0.50,
          "ls_eps": 0.0,
          "easy_margin": False
          }

In [16]:
data_transforms = {
    "train": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.),
    
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}
class HappyWhaleDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['image'].values
#         self.labels = df['individual_id_map'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = '../data/test_images-128-128/'+ self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         label = self.labels[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': img,
        }

In [123]:
df_test = pd.read_csv('../data/sample_submission.csv')

In [21]:
def train_collate(batch):

    batch_size = len(batch)
    images = [
    ]
    names = []
    for b in range(batch_size):
        if batch[b][0] is None:
            continue
        else:
            images.extend(batch[b][0])
    images = torch.stack(images, 0)
    return images

In [29]:
test_dataset = HappyWhaleDataset(df_test, transforms=data_transforms["valid"])
dataloader_test = DataLoader(test_dataset, batch_size=batch_size, num_workers=8,%%shuffle=False, pin_memory=True)

In [23]:
num_classes = 15587
model = model_whale(num_classes=num_classes, inchannels=3, model_name='senet154').cuda()

model.load_state_dict(torch.load("./weight/model_3.pth"))

In [116]:
with torch.no_grad():
    labelstrs = []
    model.eval()
    for data in tqdm(dataloader_test):
        images = data['image']
        images = images.cuda()
        _, _, outs = model(images)
        outs = torch.sigmoid(outs)
        

        
        for out in outs:
            out = torch.cat([out, torch.ones(1).cuda()*0.5], 0)
            out = out.data.cpu().numpy()
#             np.save(os.path.join(npy_dir, '{}.npy'.format(name)), out)
            top5_po = sorted(out)[-5:][::-1]
            top5 = out.argsort()[-5:][::-1]
            insert_idx = np.where(np.array(top5_po[:]) > 0.532)
            result_list = encoder.inverse_transform(top5).tolist()
            if len(insert_idx[0]) == 0:
                result_list.insert(0,'new_individual')
            else:
                result_list.insert(insert_idx[0][-1]+1, 'new_individual')
            str_top5 = ''
            labelstrs.append(' '.join(result_list[:-1]))
            
        
#             for t in top5:
#                 str_top5 += '{} '.format(id_label[t])
#             str_top5 = str_top5[:-1]
#             allnames.append(name)
#             labelstrs.append(str_top5)

100%|██████████| 1748/1748 [03:46<00:00,  7.73it/s]  


In [132]:
df_test['predictions'] = labelstrs